# 2. Decentralized Optimizaiton

## 2.1 Decentralized ADMM

Continue on the centralized ADMM algorithm. We consider the same consensus problem:
$$
    \min_{x} \sum_{i} f_i(x)
$$
Instead of relaxing $x$ into $\{x_i\}$ and forcing them to be the common value $y$, we can map it into a graph structure. More percisely, assigning $f_i$ into each vertex and relaxing the common $x$ by $\{x_i\}$ owned by each vertex. The constraint of $x_i = x_j$ can be transfered into constrain on the edge:

\begin{align}
    \min_{\{x_i\}_{i\in V},\; \{y_e\}_{e\in E} }&\;\; \sum_{i} f_i(x_i)\\
    {\rm subject\ to} &\;\; x_i - y_e=0,\;  x_j - y_e=0, \;\;\forall e={i,j} \in E
\end{align}

As long as the graph is connected, it is easy to see it is equivalent to original consenus problem.

To solve this, we consider the augmented Lagrangian and the dual variables $u_{e,i}$ for $x_i - y_e=0$ and $u_{e,j}$ for $x_j - y_e=0$. After applying the ADMM algorithm and simplifying, we obtain

\begin{align}
    x_{i}^{k+1} =& {\rm Prox}_{(\alpha |N_i|)^{-1} f_i(x_i)} (v_i^k)\\
    \alpha_{i}^{k+1} =& \frac{1}{2} x_i + \frac{1}{2|N_i|} \sum_{j\in N_i} x_j^{k+1}\\
    v_{i}^{k+1} =& v_{i}^{k} + \alpha_{i}^{k+1} - \alpha_{i}^{k}
\end{align}

The code snippet:
```python
def DecentralizedADMMStepL2(A, b, x, a, v, n_i, alpha):
    next_x = ProximalStepL2(A, b, v, n_i, alpha)
    # <-- this is the key difference. Chaning from allreduce to neighbor_allreduce --
    neighbor_weights = {r: 0.5 / n_i for r in bf.in_neighbor_ranks()}
    next_a = bf.neighbor_allreduce(
        next_x, self_weight=0.5, neighbor_weights=neighbor_weights
    )   
    next_v = v + next_a - a
    return next_x, next_a, next_v
```